## Set up the environment

In [7]:
## Set up the environment
import tensorflow as tf
print(tf.__version__)

2.9.1


In [1]:
import keras
print(keras.__version__)

2.12.0


In [2]:
# !pip install git+https://github.com/yaledhlab/vggface.git
# simply use other's edited VGGFace by git
# !pip install keras_vggface
# install from official but need to change one import in model.py

In [3]:
from keras_vggface.vggface import VGGFace
from keras import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Dropout, BatchNormalization

In [4]:
import numpy as np
train_image = np.load('train_image.npy')
train_label = np.load('train_label.npy')
test_image = np.load('test_image.npy')
test_label = np.load('test_label.npy')

In [19]:
import tensorflow_probability as tfp

def pearson_correlation(y_true, y_pred):

    correlation = tfp.stats.correlation(y_true, y_pred)
    

    return correlation

## Fine Tune VGG

There are tf.keras.applications.VGG16 (2016), keras-VGGFace 0.6 (2019) and VGGFace (2022). The 2019 one cannot be used with the latest Keras due to the compatitbility \

There are 165439116 parameters in VGGFace and 58889256 parameters in VGG16

#### VGG16 without updating weights

In [21]:
# Create a VGGFace model with the VGG16 architecture, without the final fully connected layer
vgg16_model = VGGFace(include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the VGGFace model
for layer in vgg16_model.layers:
    layer.trainable = False

last_layer = vgg16_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(256, activation='relu', name='fc6')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu', name='fc7')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
out = Dense(1, activation='linear', name='fc8')(x)
custom_vgg16_model = Model(vgg16_model.input, out)

# Compile the model with appropriate loss and optimizer
custom_vgg16_model.compile(loss='mean_squared_error', optimizer='adam', metrics=[pearson_correlation])

In [6]:
"""" By GPU
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
"""

# train_gen = DataGenerator(train_image, train_label, 16)
# test_gen = DataGenerator(test_image, test_label, 16)

# Fine-tune the model with your training dataset
# custom_vgg16_model.fit(train_gen, epochs=10, validation_data=test_gen)

'"\nfrom tensorflow.keras.utils import Sequence\nimport numpy as np   \n\nclass DataGenerator(Sequence):\n    def __init__(self, x_set, y_set, batch_size):\n        self.x, self.y = x_set, y_set\n        self.batch_size = batch_size\n\n    def __len__(self):\n        return int(np.ceil(len(self.x) / float(self.batch_size)))\n\n    def __getitem__(self, idx):\n        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]\n        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]\n        return batch_x, batch_y\n'

In [22]:
custom_vgg16_model.fit(train_image, train_label, validation_data=(test_image, test_label), batch_size=32, epochs=10)

Epoch 1/10
101/101 [==============================] - 282s 3s/step - loss: 1037.7888 - pearson_correlation: 0.6223 - val_loss: 1196.4574 - val_pearson_correlation: 0.6545
Epoch 2/10
101/101 [==============================] - 459s 5s/step - loss: 816.1761 - pearson_correlation: 0.7447 - val_loss: 785.2181 - val_pearson_correlation: 0.5688
Epoch 3/10
101/101 [==============================] - 323s 3s/step - loss: 442.4142 - pearson_correlation: 0.7854 - val_loss: 433.7262 - val_pearson_correlation: 0.6395
Epoch 4/10
101/101 [==============================] - 497s 5s/step - loss: 145.0836 - pearson_correlation: 0.7942 - val_loss: 155.2526 - val_pearson_correlation: 0.6479
Epoch 5/10
101/101 [==============================] - 295s 3s/step - loss: 41.8512 - pearson_correlation: 0.7961 - val_loss: 81.2139 - val_pearson_correlation: 0.6412
Epoch 6/10
101/101 [==============================] - 267s 3s/step - loss: 27.3041 - pearson_correlation: 0.8111 - val_loss: 56.0481 - val_pearson_correlat

In [23]:
custom_vgg16_model.save('custom_vgg16_model.h5')

#### VGG Resnet50 without updating weights

In [12]:
train_image_resnet = np.load('train_image_resnet.npy')
test_image_resnet = np.load('test_image_resnet.npy')

In [20]:

# Create a VGGFace model with the ResNet50 architecture, without the final fully connected layer
resnet50_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the VGGFace model
for layer in resnet50_model.layers:
    layer.trainable = False

# Get the output of the last layer of the ResNet50 model
last_layer = resnet50_model.get_layer('avg_pool').output

# Flatten the output of the last layer to create a 1D vector
x = Flatten(name='flatten')(last_layer)

# Add a dense layer with 256 neurons and 'relu' activation function
x = Dense(256, activation='relu', name='fc6')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Add a dense layer with 128 neurons and 'relu' activation function
x = Dense(128, activation='relu', name='fc7')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Add the final dense layer with one neuron and a linear activation function
out = Dense(1, activation='linear', name='fc8')(x)

# Create a new model that includes the ResNet50 model and the new layers
custom_resnet50_model = Model(resnet50_model.input, out)

# Compile the model
custom_resnet50_model.compile(optimizer='Adam', loss='mean_squared_error', metrics=[pearson_correlation])
history = custom_resnet50_model.fit(train_image_resnet, train_label, validation_data=(test_image_resnet, test_label), 
                                    batch_size=32, epochs=10)





Epoch 1/10
101/101 [==============================] - 79s 759ms/step - loss: 1037.7900 - pearson_correlation: 0.6160 - val_loss: 1045.1467 - val_pearson_correlation: 0.6942
Epoch 2/10
101/101 [==============================] - 80s 791ms/step - loss: 826.7362 - pearson_correlation: 0.6847 - val_loss: 788.8882 - val_pearson_correlation: 0.6708
Epoch 3/10
101/101 [==============================] - 85s 841ms/step - loss: 454.5397 - pearson_correlation: 0.7104 - val_loss: 361.4496 - val_pearson_correlation: 0.6680
Epoch 4/10
101/101 [==============================] - 85s 846ms/step - loss: 156.2869 - pearson_correlation: 0.7027 - val_loss: 112.5268 - val_pearson_correlation: 0.6226
Epoch 5/10
101/101 [==============================] - 92s 915ms/step - loss: 50.0729 - pearson_correlation: 0.7197 - val_loss: 69.1239 - val_pearson_correlation: 0.6395
Epoch 6/10
101/101 [==============================] - 82s 810ms/step - loss: 34.0699 - pearson_correlation: 0.7379 - val_loss: 56.5070 - val_pear

#### VGG SENET50 without updating weights

In [25]:
train_image_senet = np.load('train_image_resnet.npy')
test_image_senet = np.load('test_image_resnet.npy')

In [26]:
# Create a VGGFace model with the SeNet50 architecture, without the final fully connected layer
senet50_model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the VGGFace model
for layer in senet50_model.layers:
    layer.trainable = False

# Get the output of the last layer of the SeNet50 model
last_layer = senet50_model.get_layer('avg_pool').output

# Flatten the output of the last layer to create a 1D vector
x = Flatten(name='flatten')(last_layer)

# Add a dense layer with 256 neurons and 'relu' activation function
x = Dense(256, activation='relu', name='fc6')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Add a dense layer with 128 neurons and 'relu' activation function
x = Dense(128, activation='relu', name='fc7')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Add the final dense layer with one neuron and a linear activation function
out = Dense(1, activation='linear', name='fc8')(x)

# Create a new model that includes the SeNet50 model and the new layers
custom_senet50_model = Model(senet50_model.input, out)

# Compile the model
custom_senet50_model.compile(optimizer='Adam', loss='mean_squared_error', metrics=[pearson_correlation])
history = custom_senet50_model.fit(train_image_senet, train_label, validation_data=(test_image_senet, test_label), 
                                    batch_size=32, epochs=10)


104944616/104944616 [==============================] - 10s 0us/step
Epoch 1/10
101/101 [==============================] - 88s 823ms/step - loss: 1037.8566 - pearson_correlation: 0.6459 - val_loss: 978.0037 - val_pearson_correlation: 0.6919
Epoch 2/10
101/101 [==============================] - 87s 859ms/step - loss: 825.0474 - pearson_correlation: 0.7098 - val_loss: 724.0071 - val_pearson_correlation: 0.6493
Epoch 3/10
101/101 [==============================] - 98s 969ms/step - loss: 457.3849 - pearson_correlation: 0.6959 - val_loss: 352.4497 - val_pearson_correlation: 0.6803
Epoch 4/10
101/101 [==============================] - 131s 1s/step - loss: 156.2571 - pearson_correlation: 0.6901 - val_loss: 129.5309 - val_pearson_correlation: 0.6480
Epoch 5/10
101/101 [==============================] - 161s 2s/step - loss: 53.2797 - pearson_correlation: 0.6892 - val_loss: 60.4077 - val_pearson_correlation: 0.6571
Epoch 6/10
101/101 [==============================] - 88s 867ms/step - loss: 36.95

In [27]:
custom_senet50_model.save('custom_senet50_model.h5')